## Step 1: Setup & Environment Variables

Load secrets and configuration from a `.env` file for security and flexibility.

In [ ]:
# Cell 1: Setup
import os
import requests
import pandas as pd
from IPython.display import display
import json
from dotenv import load_dotenv
load_dotenv()

GRAFANA_API_KEY = os.getenv("GRAFANA_API_KEY")
GRAFANA_URL = os.getenv("GRAFANA_URL", "http://localhost:3000")
PROMETHEUS_UID = os.getenv("PROMETHEUS_UID", "your_prometheus_uid")

PROMETHEUS_QUERY_ENDPOINT = f"{GRAFANA_URL}/api/ds/query"
HEADERS = {
    "Authorization": f"Bearer {GRAFANA_API_KEY}",
    "Content-Type": "application/json"
}

## Step 2: LLM Prompt Template (Ollama Llama 3)

Generate PromQL from natural language using Llama 3 via Ollama.

In [ ]:
# Cell 2: LLM Prompt Template using Ollama Llama 3
def generate_promql(natural_query):
    prompt = f"""
You are a PromQL expert. Convert the following natural-language query into a valid PromQL expression:
Query: "{natural_query}"
PromQL:
"""
    ollama_url = "http://localhost:11434/api/generate"
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "options": {"num_predict": 100}
    }
    llm_output = ""
    import requests
    try:
        with requests.post(ollama_url, json=payload, stream=True, timeout=60) as response:
            response.raise_for_status()
            for line in response.iter_lines():
                if line:
                    try:
                        part = json.loads(line.decode())
                        llm_output += part.get("response", "")
                    except Exception:
                        pass
    except Exception as e:
        print("Ollama connection or generation error:", e)
    # Extract PromQL from output
    promql_start = llm_output.find('PromQL:')
    if promql_start != -1:
        promql = llm_output[promql_start + len('PromQL:'):].strip().split('\n')[0]
    else:
        promql = llm_output.strip().split('\n')[0]
    return promql

## Step 3: Query Grafana with PromQL

Send the generated PromQL to Grafana using API credentials from `.env`.

In [ ]:
# Cell 3: Query Grafana with PromQL
def query_grafana(promql, start="now-7d", end="now"):
    payload = {
        "queries": [{
            "refId": "A",
            "datasource": {"type": "prometheus", "uid": PROMETHEUS_UID},
            "expr": promql,
            "interval": "1m",
            "format": "time_series",
            "instant": False
        }],
        "range": {
            "from": start,
            "to": end
        }
    }
    response = requests.post(PROMETHEUS_QUERY_ENDPOINT, headers=HEADERS, data=json.dumps(payload))
    data = response.json()
    return data

In [ ]:
# Cell 4: Display Results
def display_results(data):
    try:
        results = data['results']['A']['frames'][0]['data']['values']
        timestamps = results[0]
        values = results[1]
        df = pd.DataFrame({'Time': timestamps, 'Value': values})
        display(df)
    except Exception as e:
        print("Error parsing Grafana response:", e)

In [ ]:
# Cell 5: Run Full Flow
natural_query = "Show me last week's error rate"
promql = generate_promql(natural_query)
print("Generated PromQL:", promql)

grafana_data = query_grafana(promql)
display_results(grafana_data)